In [6]:
%matplotlib inline

import os
import textwrap
import math

import datetime as dt
import pandas as pd

import matplotlib        as mpl
import matplotlib.pyplot as plt

import numpy             as np
import tables            as tb
import seaborn as sns

#import invisible_cities.core.core_functions as coref
#import invisible_cities.core.fit_functions  as fitf

from   scipy.optimize                          import curve_fit
from   scipy                                   import stats

#from   invisible_cities.io.dst_io import df_writer

# Formatting options
pd.options.display.max_columns = None
mpl.rcParams.update({'font.size': 18})
mpl.rcParams['image.cmap'] = 'Greys'
mpl.rcParams['patch.force_edgecolor'] = False
mpl.rcParams['patch.facecolor'] = 'gray'
hargs = {'histtype': 'stepfilled', 'edgecolor': 'black', 'facecolor': 'gray'}

# Directory to which figures will be stored
#save_dir = "Plots Iniciales"
#ftype = "pdf"
#if(not os.path.isdir(save_dir)):
#    print("Creating directory",save_dir)
#    os.mkdir(save_dir)

In [7]:
evtfnames_data_with_hits = [r"/Users/diego/TFG/Mis movidas de Python/ntuple_combined_7746_con_hits.h5"] 
evtfnames_MC   = ["/Users/diego/Dropbox/NEXT/ntuple_combined_MC_6206.h5",
              "/Users/diego/Dropbox/NEXT/ntuple_combined_MC_6207.h5",
              "/Users/diego/Dropbox/NEXT/ntuple_combined_MC_6208.h5",]

In [ ]:
cdf = None
hits = None

for ifile,fname in enumerate(evtfnames_data_with_hits): #SE QUEREMOS VER OS RESULTADOS PARA OS DATOS: *evtfnames_data*

    print("Adding file {} ...".format(fname))

    ftbl = tb.open_file(fname, 'r')
    
    fsummary = ftbl.root.Summary.Events
    fdst = ftbl.root.DST.Events
    ftracks = ftbl.root.Tracking.Tracks
    fhits = ftbl.root.CHITS.highTh
    
    # Read the tables into DataFrames.
    to_df = pd.DataFrame.from_records
    df_summary = to_df(fsummary.read())
    df_tracks = to_df(ftracks.read())
    df_dst = to_df(fdst.read())
    df_hits = to_df(fhits.read())
    
    # Drop duplicates in the KDST, selecting the S1 and S2 with highest energy.
    # df_dst1 = df_dst.sort_values(['event','S1e','S2e'])[['event','S2e','S1e','nS1','nS2']].drop_duplicates('event',keep='last')
    
    # Drop duplicates in the TDST, selecting the longest tracks.
    # df_tracks = df_tracks.sort_values(['event','length','energy']).drop_duplicates('event',keep='last')
    
    # Merge into a single dataframe.
    df_merge = pd.merge(df_summary, df_dst, on = 'event', how='left')
    df_merge = pd.merge(df_merge, df_tracks, on = 'event', how='left')
    
    # Concatenate with previous runs.
    if(cdf is None):
        cdf = df_merge
    else:
        cdf = cdf.append(df_merge)
    
    if (hits is None):
        hits = df_hits
    else:
        hits = df_hits.append(df_hits)
        
    ftbl.close()

A_evtnum    = cdf['event'].values
A_maxtrklen = cdf['length'].values
A_maxtrkenergy = cdf['energy'].values
A_ntrks     = cdf['evt_ntrks'].values
A_nhits     = cdf['evt_nhits'].values
A_Ec        = cdf['evt_energy'].values
A_xavg      = cdf['evt_x_avg'].values
A_yavg      = cdf['evt_y_avg'].values
A_zavg      = cdf['evt_z_avg'].values
A_ravg      = cdf['evt_r_avg'].values
A_xmin      = cdf['evt_x_min'].values
A_ymin      = cdf['evt_y_min'].values
A_zmin      = cdf['evt_z_min'].values
A_xmax      = cdf['evt_x_max'].values
A_ymax      = cdf['evt_y_max'].values
A_zmax      = cdf['evt_z_max'].values
A_rmin      = cdf['evt_r_min'].values
A_rmax      = cdf['evt_r_max'].values
A_nS1       = cdf['nS1'].values
A_S2e       = cdf['S2e'].values

print("{} total events for Summary df".format(len(A_evtnum)))
print("{} total events for Hits df".format(len(hits['event'].values)))

cdf.head(10)

Adding file /Users/diego/TFG/Mis movidas de Python/ntuple_combined_7746_con_hits.h5 ...


# Porcentaxe Número de S2 

Hago un histograma que me permita ver el número de pulsos S2 que hay en los eventos que estamos analizando. Podemos ver que predominan los eventos con nS2 = 1, pero también aparecen algunos con 2, 3 o incluso 4 pulsos S2.

|    S2           |    %    | 
| --------------- | ------- |
| 1               | 78.315% |
| 2               | 18.097% |
| 3               | 3.048%  |
| 4               | < 1%    |
| 5, 6, 7, 8 e 9  | < 1%    |

In [ ]:
bins = np.arange(start=1, stop=11) - 0.5

#HISTOGRAMA
n, bins, patches = plt.hist(cdf.nS2, bins, edgecolor='black', width=0.8, density = True)
plt.xticks(np.arange(start=1, stop=10))
plt.xlabel('Nº de eventos S2')
plt.ylabel('Contas')

#CREO LOS PORCENTAJES
#x = stats.describe(cdf.nS2)
#a = []
#for i in range(len(n)):
#    a.append((n[i]/x[0])*100)
    
print('INFORMACIÓN XENERAL SOBRE S2')
print(stats.describe(cdf.nS2.dropna()))
#print('')
#print('COUNTS')
#print('1 S2: {:.3f}%; 2 S2: {:.3f}%; 3 S2: {:.3f}%; 4 S2: {:.3f}%; 5 S2: {:.3f}%;'
#      ' 6 S2: {:.3f}%; 7 S2: {:.3f}%; 8 S2: {:.6f}%; 9 S2: {:.6f}%'.format(a[0],a[1],a[2],a[3],a[4],
#                                                                                    a[5],a[6],a[7],a[8]))

# Nº de S2 == 1

In [ ]:
#PRIMER CORTE
one_S2 = cdf[(cdf.nS2 == 1)]

bins = np.arange(start=1, stop=9) - 0.5
n, bins, patches = plt.hist(one_S2.numb_of_tracks, bins, edgecolor='black', width=0.8)
plt.xticks(np.arange(start=1, stop=8))
plt.xlabel('Nº de trazas para nS2 == 1')
plt.ylabel('Contas')

x = stats.describe(one_S2.numb_of_tracks.dropna())
a = []
for i in range(len(n)):
    a.append((n[i]/x[0])*100)

b = np.sum(a[1:])

print('INFORMACIÓN XENERAL SOBRE AS TRAZAS')
print(x)
print('')
print('CONTAS')
print('nS2 == 1 & nTr == 1 --> {:.3f}% \nnS2 == 1 & nTr == 2 --> {:.3f}% \nnS2 == 1 & nTr == 3 --> {:.3f}%'
      '\nnS2 == 1 & nTr == 4 --> {:.3f}%'.format(a[0], a[1], a[2], a[3]))

# Nº de S2 == 2

In [ ]:
two_S2 = cdf[(cdf.nS2 == 2)]

bins = np.arange(start=1, stop=9) - 0.5
n, bins, patches = plt.hist(two_S2.numb_of_tracks, bins, edgecolor='black', width=0.8)
plt.xticks(np.arange(start=1, stop=8))
plt.xlabel('Nº de trazas para nS2 == 2')
plt.ylabel('Contas')

x = stats.describe(two_S2.numb_of_tracks)
a = []
for i in range(len(n)):
    a.append((n[i]/x[0])*100)

b = np.sum(a) - a[1]

print('INFORMACIÓN XENERAL SOBRE AS TRAZAS')
print(x)
print('')
print('CONTAS')
print('nS2 == 2 & nTr == 1 --> {:.3f}% \nnS2 == 2 & nTr == 2 --> {:.3f}%, \nnS2 == 2 & nTr == 3 --> {:.3f}%'
      '\nnS2 == 2 & nTr == 4 --> {:.3f}%, \nnS2 == 2 & nTr == 5 --> {:.3f}%, \nnS2 == 2 & nTr == 6 --> {:.3f}%'
      '\nnS2 == 2 & nTr == 7 --> {:.3f}%'.format(a[0], a[1], a[2], a[3], a[4], a[5], a[6]))

# Nº de S2 > 2

In [ ]:
large_S2 = cdf[(cdf.nS2 > 2)]

bins = np.arange(start=1, stop=10) - 0.5
n, bins, patches = plt.hist(large_S2.numb_of_tracks, bins, edgecolor='black', width=0.8)
plt.xticks(np.arange(start=1, stop=9))
plt.xlabel('Nº de trazas para nS2 > 2')
plt.ylabel('Contas')


e = 0
d = 0

for i, j in zip(large_S2.nS2,large_S2.numb_of_tracks):
    if i == j:
        e += 1
    else:
        d += 1
        
f = e/len(large_S2.numb_of_tracks)*100
g = d/len(large_S2.numb_of_tracks)*100


print('INFORMACIÓN XENERAL SOBRE AS TRAZAS')
print(stats.describe(large_S2.numb_of_tracks.dropna()))
print('')
print('CONTAS')
print('nS2 > 2 & nTr == nS2 --> {:.3f}% \nnS2 > 2 & nTr != nS2 --> {:.3f}%'.format(f, g))

# Resultados Para Datos *Monte Carlo*

Presentamos en esta tabla los resultados del análisis del número de S2 frente al número de trazas, que nos proporciona resultados en un principio desconcertantes como, por ejemplo, que para eventos en los que nS2 == 2 tengamos exclusivamente una traza.

|   nS2 \ nTr | 1       | 2       | 3        | 4      | 5    | 6    | 7    |  
| ----------- | ------- | ------- | -------- | ------ | ---- | ---- | ---- |
| 1           | 77.795% | 17.888% | 3.614%   | < 1%   | < 1% | < 1% | < 1% |
| 2           | 16.502% | 65.885% | 14. 833% | 2.409% | < 1% | < 1% | < 1% |

A mayores, tenemos como generalización de la tabla anterior otra en la que establecemos una relación más general. Esta vez comparamos los eventos en los que coinciden o difieren nS2 y el número de trazas para ese mismo evento (para nS2 > 2).

| nS2 \ nTr   | == nS2  | != nS2  |
| ----------- | ------- | ------- |
| > 2         | 42.511% | 57.489% |

# Resultados Para Datos *Reales*

Presentamos en esta tabla los resultados del análisis del número de S2 frente al número de trazas, que nos proporciona resultados en un principio desconcertantes como, por ejemplo, que para eventos en los que nS2 == 2 tengamos exclusivamente una traza.

| nS2 \ nTr   | 1       | 2       | 3        | 4      | 5    | 6    | 7    |  
| ----------- | ------- | ------- | -------- | ------ | ---- | ---- | ---- |
| 1           | 94.832% |  4.672% | < 1%     | < 1%   | < 1% | < 1% | < 1% |
| 2           | 34.744% | 60.284% | 4.513%   | < 1%   | < 1% | < 1% | < 1% |

A mayores, tenemos como generalización de la tabla anterior otra en la que establecemos una relación más general. Esta vez comparamos los eventos en los que coinciden o difieren nS2 y el número de trazas para ese mismo evento (para nS2 > 2).

| nS2 \ nTr   | == nS2  | != nS2  |
| ----------- | ------- | ------- |
| > 2         | 39.575% | 60.425% |

# nTr != nS2

Queremos hilar un poco fino que antes, de manera que ahora subdividimos este apartado ya existente en otros dos. Cuando el número de trazas es **mayor** que el número de pulsos S2, y cuando este es **menor**.

In [ ]:
large_S2 = cdf[(cdf.nS2 > 2)]

a = 0 #acumulador para ntr > ns2
b = 0 #acumulador para ntr < ns2
non_equal = []

for nS2, ntr in zip(large_S2.nS2,large_S2.numb_of_tracks):
    if ntr != nS2:
        non_equal.append(ntr)
        if ntr > nS2:
            a += 1
        else:
            b += 1    

#Porcentaxes:
c = a/len(non_equal)*100
d = b/len(non_equal)*100

print(a,b,c,d)

Para os eventos en que o número de trazas que aparecen difire do número de pulsos S2, vemos que predominan (evidentemente(?)) os eventos en que o número de trazas é menor que o número de pulsos. Aínda así, hai unha importante proporción de eventos nos que nTr supera a nS2, como vemos na táboa. Isto todavía non somos capaces de explicalo.

|               | nTr > nS2 | nTr < nS2 |
| ------------- | --------- | --------- |
| %             | 7.139%    | 92.862%   |
| nº de eventos | 932       | 12008     |